In [1]:
import pandas as pd

# Ruta del archivo GTF
ruta_archivo = 'hg38.ncbiRefSeq.gtf'

# Columnas que esperamos en el DataFrame
columnas = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']

# Leer el archivo GTF en un DataFrame de pandas
df = pd.read_csv(ruta_archivo, sep='\t', comment='#', header=None, names=columnas)

# Ajustar la anchura de la columna para que se muestren todas las columnas
pd.set_option('display.max_colwidth', None)

# Visualizar el DataFrame completo
df = df[df['feature'] == 'transcript'].reset_index(drop = True)

In [2]:
import re

# Definir una función para extraer los valores de cada campo usando expresiones regulares
def extraer_valor(cadena, campo):
    coincidencia = re.search(f'{campo} "(.*?)"', cadena)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'atributos' para extraer los valores de cada campo
df['gene_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'gene_id'))
df['transcript_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'transcript_id'))
df['gene_name'] = df['attributes'].apply(lambda x: extraer_valor(x, 'gene_name'))


In [3]:
mask = df['transcript_id'].str.match('.*NM.*|.*NR.*')

# Aplicar la máscara al DataFrame para filtrar los valores
df_filtrado = df[mask]

In [4]:
# Lista de cromosomas
cromosomas = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
              'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM']

# Filtrar el DataFrame para mantener solo los cromosomas de interés
df_filtrado = df_filtrado[df_filtrado['seqname'].isin(cromosomas)]


In [5]:
# Calcular el máximo de la columna 'end' para cada grupo 'gene_id'
maximos_por_gene = df_filtrado.groupby(['gene_id','seqname'])['end'].max().reset_index()

# Renombrar la columna 'end' como 'pos_max'
maximos_por_gene = maximos_por_gene.rename(columns={'end': 'pos_max'})

# Calcular el mínimo de la columna 'start' para cada grupo 'gene_id'
minimos_por_gene = df_filtrado.groupby(['gene_id','seqname'])['start'].min().reset_index()

# Renombrar la columna 'start' como 'pos_min'
minimos_por_gene = minimos_por_gene.rename(columns={'start': 'pos_min'})


In [6]:
# Suponiendo que tienes los DataFrames df_maximos_por_gene y df_minimos_por_gene que quieres combinar

# Combinar los DataFrames a lo ancho por la columna 'gene_id'
df_combinado = pd.merge(minimos_por_gene, maximos_por_gene, on=['gene_id','seqname'])


In [7]:
df_combinado

,gene_id,seqname,pos_min,pos_max
0,A1BG,chr19,58345183,58353492
1,A1BG-AS1,chr19,58351970,58355183
2,A1CF,chr10,50799409,50885627
3,A2M,chr12,9067708,9116229
4,A2M-AS1,chr12,9065177,9068055
...,...,...,...,...
28770,ZYG11A,chr1,52842760,52894998
28771,ZYG11B,chr1,52726453,52827336
28772,ZYX,chr7,143381345,143391111
28773,ZZEF1,chr17,4004445,4143030


In [8]:
# Contar cuántos duplicados hay en la columna 'gene_id'
numero_de_duplicados = df_combinado.duplicated(subset=['gene_id']).sum()

# Imprimir el número de duplicados
print(numero_de_duplicados)

27


In [9]:
# Filtrar duplicados basados en la columna 'gene_id'
duplicados_por_gene_id = df_combinado[df_combinado.duplicated(subset=['gene_id'], keep=False)]

# Imprimir el DataFrame con los duplicados por 'gene_id'
duplicados_por_gene_id

,gene_id,seqname,pos_min,pos_max
583,AKAP17A,chrX,1591604,1602520
584,AKAP17A,chrY,1591604,1602520
1346,ASMT,chrX,1595455,1643081
1347,ASMT,chrY,1595455,1643081
1348,ASMTL,chrX,1403139,1453756
1349,ASMTL,chrY,1403139,1453756
1350,ASMTL-AS1,chrX,1400531,1415421
1351,ASMTL-AS1,chrY,1400531,1415421
3065,CD99,chrX,2691295,2741309
3066,CD99,chrY,2691295,2741309


In [10]:
df_combinado = df_combinado.drop_duplicates('gene_id')

# Esta parte solo hay que hacerla una vez para conseguir un excel mejorado con los símbolos de los genes que interactúan entre ellos

In [162]:
# Ruta del archivo CSV
ruta_archivo_csv = 'genes_id_3.csv'

# Cargar el archivo CSV en un DataFrame
df_genes_h = pd.read_csv(ruta_archivo_csv)

In [163]:
# Define la función para extraer el Official Symbol
def extraer_official_symbol(cadena):
    coincidencia = re.search(r'Official Symbol: (\w+-\w+|\w+)', cadena)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplica la función a cada fila de la columna 'descripcion' para extraer el "Official Symbol"
df_genes_h['Symbol'] = df_genes_h['secuencia'].apply(extraer_official_symbol)


In [164]:
def extraer_symbol_alternativo(cadena):
    coincidencia = re.search(r'1. (\w+)', cadena)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'descripcion' para extraer el "Official Symbol"
df_genes_h['Symbol_alt'] = df_genes_h['secuencia'].apply(extraer_symbol_alternativo)

In [165]:
df_genes_h['Symbol'].fillna(df_genes_h['Symbol_alt'], inplace=True)

In [166]:
def extraer_numero_cromosoma(informacion):
    coincidencia = re.search(r'Chromosome: (\w+)', informacion)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'informacion' para extraer el número del cromosoma
df_genes_h['numero_cromosoma'] = df_genes_h['secuencia'].apply(extraer_numero_cromosoma)

In [167]:
def extraer_numero_cromosoma_alt(informacion):
    coincidencia = re.search(r'Chromosome (\w+)', informacion)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'informacion' para extraer el número del cromosoma
df_genes_h['numero_cromosoma_alt'] = df_genes_h['secuencia'].apply(extraer_numero_cromosoma)

In [168]:
df_genes_h['numero_cromosoma'].fillna(df_genes_h['numero_cromosoma_alt'], inplace=True)

In [169]:
# Especifica la ruta y el nombre del archivo CSV
ruta_archivo_csv = 'genes_nuevo.csv'

# Convierte el DataFrame a CSV
df_genes_h.to_csv(ruta_archivo_csv, index=False)

# El argumento index=False evita que se guarde el índice del DataFrame como una columna adicional en el archivo CSV

# A partir de aquí comienza la segunda parte en la que vamos a cruzar los dos dataframes 

In [11]:
# Ruta del archivo CSV
ruta_archivo_csv = 'Excel_genes_bueno.xlsx'

# Cargar el archivo CSV en un DataFrame
df_genes = pd.read_excel(ruta_archivo_csv)

In [12]:
df_genes.drop(['secuencia','Symbol_alt','numero_cromosoma_alt','id_nucleotidos','anotacion'],axis = 1,inplace = True)

In [13]:
registros_sin_symbol = df_genes[df_genes['Symbol'].isna()]

In [14]:
registros_sin_symbol

,id,Symbol,numero_cromosoma


In [15]:
df_genes.count()

id                  20050
Symbol              20050
numero_cromosoma    20042
dtype: int64

In [16]:
import pandas as pd

# Cruzar los DataFrames por los campos especificados
df_merged = pd.merge(df_genes, df_combinado, left_on='Symbol', right_on='gene_id', how = 'left')

In [17]:
df_merged_left = df_merged[df_merged['gene_id'].isna()]
df_merged_inner = df_merged[~df_merged['gene_id'].isna()]

In [18]:
df_merged_left

,id,Symbol,numero_cromosoma,gene_id,seqname,pos_min,pos_max
120,171,AFD1,9,NaN,NaN,NaN,NaN
144,206,AK4P1,17,NaN,NaN,NaN,NaN
299,406,BMAL1,11,NaN,NaN,NaN,NaN
313,425,ARVD4,2,NaN,NaN,NaN,NaN
398,574,BAGE,21,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
20014,102723170,IGHV3-64D,14,NaN,NaN,NaN,NaN
20024,102724334,LOC102724334,21,NaN,NaN,NaN,NaN
20038,103504728,MIR3687-2,21,NaN,NaN,NaN,NaN
20041,104355133,ABHD15-AS1,17,NaN,NaN,NaN,NaN


In [19]:
df_merged_inner

,id,Symbol,numero_cromosoma,gene_id,seqname,pos_min,pos_max
0,1,A1BG,19,A1BG,chr19,58345183.0,58353492.0
1,2,A2M,12,A2M,chr12,9067708.0,9116229.0
2,9,NAT1,8,NAT1,chr8,18170467.0,18223689.0
3,10,NAT2,8,NAT2,chr8,18391282.0,18401218.0
4,12,SERPINA3,14,SERPINA3,chr14,94612391.0,94624053.0
...,...,...,...,...,...,...,...
20044,105375355,UPK3B,7,UPK3B,chr7,76510552.0,76516522.0
20046,105377105,FLNB-AS1,3,FLNB-AS1,chr3,58162547.0,58170635.0
20047,105378803,LRRC53,1,LRRC53,chr1,74469376.0,74512611.0
20048,107126285,SEPTIN14P20,20,SEPTIN14P20,chr20,64290385.0,64303354.0


In [20]:
df_merged_inner['pos_min'] = df_merged_inner['pos_min'].astype(int)
df_merged_inner['pos_max'] = df_merged_inner['pos_max'].astype(int)

C:\Users\mojoy\AppData\Local\Temp\ipykernel_6368\2470853460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_inner['pos_min'] = df_merged_inner['pos_min'].astype(int)
C:\Users\mojoy\AppData\Local\Temp\ipykernel_6368\2470853460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_inner['pos_max'] = df_merged_inner['pos_max'].astype(int)


In [22]:
'''Creo que estos genes que no están en la base de datos del GTF a la fecha en la que se hizo su recogida. La fecha de la
base de datos es de 2022 y por ejemplo para los genes AK4P1 y BMAL1 sí que tienen una secuencia de nucleótidos asociada pero que
se ha hecho en 2023.'''

'Creo que estos genes que no están en la base de datos del GTF a la fecha en la que se hizo su recogida. La fecha de la\nbase de datos es de 2022 y por ejemplo para los genes AK4P1 y BMAL1 sí que tienen una secuencia de nucleótidos asociada pero que\nse ha hecho en 2023.'

# Vamos a ver si el problema persiste para las otras bases de datos

In [289]:
import pandas as pd

# Ruta del archivo GTF
ruta_archivo = 'hg38.refGene.gtf'

# Columnas que esperamos en el DataFrame
columnas = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']

# Leer el archivo GTF en un DataFrame de pandas
df = pd.read_csv(ruta_archivo, sep='\t', comment='#', header=None, names=columnas)

# Ajustar la anchura de la columna para que se muestren todas las columnas
pd.set_option('display.max_colwidth', None)

# Visualizar el DataFrame completo
df = df[df['feature'] == 'transcript'].reset_index(drop = True)

In [290]:
import re

# Definir una función para extraer los valores de cada campo usando expresiones regulares
def extraer_valor(cadena, campo):
    coincidencia = re.search(f'{campo} "(.*?)"', cadena)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'atributos' para extraer los valores de cada campo
df['gene_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'gene_id'))
df['transcript_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'transcript_id'))
df['gene_name'] = df['attributes'].apply(lambda x: extraer_valor(x, 'gene_name'))

In [291]:
mask = df['transcript_id'].str.match('.*NM.*|.*NR.*')

# Aplicar la máscara al DataFrame para filtrar los valores
df_filtrado = df[mask]

In [292]:
# Lista de cromosomas
cromosomas = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
              'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM']

# Filtrar el DataFrame para mantener solo los cromosomas de interés
df_filtrado = df_filtrado[df_filtrado['seqname'].isin(cromosomas)]


In [293]:
# Calcular el máximo de la columna 'end' para cada grupo 'gene_id'
maximos_por_gene = df_filtrado.groupby(['gene_id','seqname'])['end'].max().reset_index()

# Renombrar la columna 'end' como 'pos_max'
maximos_por_gene = maximos_por_gene.rename(columns={'end': 'pos_max'})

# Calcular el mínimo de la columna 'start' para cada grupo 'gene_id'
minimos_por_gene = df_filtrado.groupby(['gene_id','seqname'])['start'].min().reset_index()

# Renombrar la columna 'start' como 'pos_min'
minimos_por_gene = minimos_por_gene.rename(columns={'start': 'pos_min'})


In [294]:
# Suponiendo que tienes los DataFrames df_maximos_por_gene y df_minimos_por_gene que quieres combinar

# Combinar los DataFrames a lo ancho por la columna 'gene_id'
df_combinado = pd.merge(minimos_por_gene, maximos_por_gene, on=['gene_id','seqname'])


In [295]:
df_combinado

,gene_id,seqname,pos_min,pos_max
0,A1BG,chr19,58345183,58353492
1,A1BG-AS1,chr19,58351970,58355183
2,A1CF,chr10,50799409,50885675
3,A2M,chr12,9067708,9116229
4,A2M-AS1,chr12,9065177,9068055
...,...,...,...,...
28398,ZYG11A,chr1,52842760,52894995
28399,ZYG11B,chr1,52726453,52827336
28400,ZYX,chr7,143381345,143391111
28401,ZZEF1,chr17,4004445,4143030


In [296]:
# Contar cuántos duplicados hay en la columna 'gene_id'
numero_de_duplicados = df_combinado.duplicated(subset=['gene_id']).sum()

# Imprimir el número de duplicados
print(numero_de_duplicados)

153


In [297]:
# Filtrar duplicados basados en la columna 'gene_id'
duplicados_por_gene_id = df_combinado[df_combinado.duplicated(subset=['gene_id'], keep=False)]

# Imprimir el DataFrame con los duplicados por 'gene_id'
duplicados_por_gene_id

,gene_id,seqname,pos_min,pos_max
568,AKAP17A,chrX,1591604,1602518
569,AKAP17A,chrY,1591604,1602518
1316,ASMT,chrX,1595455,1643081
1317,ASMT,chrY,1595455,1643081
1318,ASMTL,chrX,1403139,1453762
...,...,...,...,...
27213,WASIR1,chrY,57201084,57203350
27394,XGY2,chrX,2752296,2774996
27395,XGY2,chrY,2752296,2774996
27504,ZBED1,chrX,2486414,2500976


In [298]:
df_combinado = df_combinado.drop_duplicates('gene_id')

In [299]:
# Ruta del archivo CSV
ruta_archivo_csv = 'Excel_genes_bueno.xlsx'

# Cargar el archivo CSV en un DataFrame
df_genes = pd.read_excel(ruta_archivo_csv)

In [300]:
df_genes.drop(['secuencia','Symbol_alt','numero_cromosoma_alt','id_nucleotidos','anotacion'],axis = 1,inplace = True)

In [301]:
import pandas as pd

# Cruzar los DataFrames por los campos especificados
df_merged = pd.merge(df_genes, df_combinado, left_on='Symbol', right_on='gene_id', how = 'left')

In [302]:
df_merged_left = df_merged[df_merged['gene_id'].isna()]
df_merged_inner = df_merged[~df_merged['gene_id'].isna()]

In [303]:
df_merged_left

,id,Symbol,numero_cromosoma,gene_id,seqname,pos_min,pos_max
120,171,AFD1,9,NaN,NaN,NaN,NaN
144,206,AK4P1,17,NaN,NaN,NaN,NaN
299,406,BMAL1,11,NaN,NaN,NaN,NaN
313,425,ARVD4,2,NaN,NaN,NaN,NaN
568,802,CALM1P1,X,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
20032,102724652,LOC102724652,21,NaN,NaN,NaN,NaN
20038,103504728,MIR3687-2,21,NaN,NaN,NaN,NaN
20041,104355133,ABHD15-AS1,17,NaN,NaN,NaN,NaN
20045,105376906,ERVK-19,19,NaN,NaN,NaN,NaN


In [310]:
df_malos = df[df['gene_id'] == 'BMAL1']

In [304]:
'''Con esta base de datos obtenemos más registros que no nos cruzan.'''

'Con esta base de datos obtenemos más registros que no nos cruzan.'

# Vamos a comprobar la última

In [282]:
import pandas as pd

# Ruta del archivo GTF
ruta_archivo = 'hg38.knownGene.gtf'

# Columnas que esperamos en el DataFrame
columnas = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']

# Leer el archivo GTF en un DataFrame de pandas
df = pd.read_csv(ruta_archivo, sep='\t', comment='#', header=None, names=columnas)

# Ajustar la anchura de la columna para que se muestren todas las columnas
pd.set_option('display.max_colwidth', None)

# Visualizar el DataFrame completo
df = df[df['feature'] == 'transcript'].reset_index(drop = True)

In [284]:
import re

# Definir una función para extraer los valores de cada campo usando expresiones regulares
def extraer_valor(cadena, campo):
    coincidencia = re.search(f'{campo} "(.*?)"', cadena)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Aplicar la función a cada fila de la columna 'atributos' para extraer los valores de cada campo
df['gene_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'gene_id'))
df['transcript_id'] = df['attributes'].apply(lambda x: extraer_valor(x, 'transcript_id'))

In [286]:
# Lista de cromosomas
cromosomas = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
              'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM']

# Filtrar el DataFrame para mantener solo los cromosomas de interés
df_filtrado = df[df['seqname'].isin(cromosomas)]


In [288]:
df

,seqname,source,feature,start,end,score,strand,frame,attributes,gene_id,transcript_id
0,chr1,knownGene,transcript,11869,14409,.,+,.,"gene_id ""ENST00000456328.2""; transcript_id ""ENST00000456328.2"";",ENST00000456328.2,ENST00000456328.2
1,chr1,knownGene,transcript,17369,17436,.,-,.,"gene_id ""ENST00000619216.1""; transcript_id ""ENST00000619216.1"";",ENST00000619216.1,ENST00000619216.1
2,chr1,knownGene,transcript,29554,31097,.,+,.,"gene_id ""ENST00000473358.1""; transcript_id ""ENST00000473358.1"";",ENST00000473358.1,ENST00000473358.1
3,chr1,knownGene,transcript,30267,31109,.,+,.,"gene_id ""ENST00000469289.1""; transcript_id ""ENST00000469289.1"";",ENST00000469289.1,ENST00000469289.1
4,chr1,knownGene,transcript,30366,30503,.,+,.,"gene_id ""ENST00000607096.1""; transcript_id ""ENST00000607096.1"";",ENST00000607096.1,ENST00000607096.1
...,...,...,...,...,...,...,...,...,...,...,...
274026,chrY_KZ208924v1_fix,knownGene,transcript,58534,60100,.,-,.,"gene_id ""ENST00000643054.1""; transcript_id ""ENST00000643054.1"";",ENST00000643054.1,ENST00000643054.1
274027,chrY_KZ208924v1_fix,knownGene,transcript,82985,95067,.,+,.,"gene_id ""ENST00000647527.1""; transcript_id ""ENST00000647527.1"";",ENST00000647527.1,ENST00000647527.1
274028,chrY_KZ208924v1_fix,knownGene,transcript,98305,102177,.,-,.,"gene_id ""ENST00000644360.1""; transcript_id ""ENST00000644360.1"";",ENST00000644360.1,ENST00000644360.1
274029,chrY_KZ208924v1_fix,knownGene,transcript,117629,119279,.,+,.,"gene_id ""ENST00000645951.1""; transcript_id ""ENST00000645951.1"";",ENST00000645951.1,ENST00000645951.1


In [ ]:
'''Con esta base de datos no tenemos un campo por el que cruzar con la nuestra.'''

# Cruzamos con nuestra tabla de interacciones

In [11]:
'''Vamos a cargar la tabla de interacciones'''

'Vamos a cargar la tabla de interacciones'

In [40]:
# Importa las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, countDistinct, explode, split, size
# Crea una instancia de SparkSession
spark = SparkSession.builder.appName("Ejemplo").getOrCreate()

path_4 = 'interactions\interactions.txt'
df_int = spark.read.csv(path_4, header=True, inferSchema=True, sep='\t')

In [41]:
df_int_homo = df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606))

In [42]:
df_int_homo_filtrado = df_int_homo.select('gene_id','interactant_id','pubmed_id_list')
df_int_homo_filtrado = df_int_homo_filtrado.drop_duplicates()
df_int_homo_filtrado = df_int_homo_filtrado.toPandas()

In [43]:
df_int_homo_filtrado = df_int_homo_filtrado.sort_values(by='gene_id')

In [44]:
df_int_homo_filtrado.head()

,gene_id,interactant_id,pubmed_id_list
0,1,2886,21900206
772216,1,80854,21988832
770099,1,84236,33961781
770098,1,148581,33961781
769570,1,5655,33961781


In [45]:
df_1 = pd.merge(df_int_homo_filtrado, df_merged_inner[['id', 'Symbol', 'seqname', 'pos_min', 'pos_max']], left_on='gene_id', right_on='id', how='left')

In [46]:
df_1_inner = df_1[~df_1['seqname'].isnull()]

# Estos registros que no cruzan como ya he comentado antes, es porque la secuencia se ha incorporado más tarde a la base de datos.
df_1_left = df_1[df_1['seqname'].isnull()]

In [47]:
df_comprobacion = pd.merge(df_1_left, df_merged_left[['id', 'Symbol', 'seqname', 'pos_min', 'pos_max']], left_on='gene_id', right_on='id', how='left')

In [50]:
df_comprobacion.drop_duplicates('gene_id')

# Comprobamos que los id son los que ya sabíamos que no tenían secuencia asociada.

,gene_id,interactant_id,pubmed_id_list,id_x,Symbol_x,seqname_x,pos_min_x,pos_max_x,id_y,Symbol_y,seqname_y,pos_min_y,pos_max_y
0,171,3265,30442766,NaN,NaN,NaN,NaN,NaN,171,AFD1,NaN,NaN,NaN
15,206,10212,32393512,NaN,NaN,NaN,NaN,NaN,206,AK4P1,NaN,NaN,NaN
16,406,3159,30021884,NaN,NaN,NaN,NaN,NaN,406,BMAL1,NaN,NaN,NaN
120,425,10618,35696571,NaN,NaN,NaN,NaN,NaN,425,ARVD4,NaN,NaN,NaN
121,574,3191,28611215,NaN,NaN,NaN,NaN,NaN,574,BAGE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4737,653319,728116,32296183,NaN,NaN,NaN,NaN,NaN,653319,MATCAP1,NaN,NaN,NaN
4738,653440,100287171,33961781,NaN,NaN,NaN,NaN,NaN,653440,WASH6P,NaN,NaN,NaN
4739,653639,728655,32572027,NaN,NaN,NaN,NaN,NaN,653639,LYPLA2P1,NaN,NaN,NaN
4740,100132285,100132285,11172020,NaN,NaN,NaN,NaN,NaN,100132285,KIR2DS2,NaN,NaN,NaN


In [51]:
# Nos quitamos las interacciones donde no se indica el otro gen
df_1_inner = df_1_inner[~(df_1_inner['interactant_id'] == '-')]
df_1_inner['interactant_id'] = df_1_inner['interactant_id'].astype(int)

In [52]:
# Ahora vamos a cruzar el df_1_inner con la base de datos de las posiciones para obtener la secuencia del gen que interactúa
df_2 = pd.merge(df_1_inner, df_merged_inner[['id', 'Symbol', 'seqname', 'pos_min', 'pos_max']], left_on='interactant_id', right_on='id', how='left')

In [53]:
df_2_inner = df_2[~df_2['seqname_y'].isnull()]
df_2_left = df_2[df_2['seqname_y'].isnull()]

In [54]:
df_2_inner

,gene_id,interactant_id,pubmed_id_list,id_x,Symbol_x,seqname_x,pos_min_x,pos_max_x,id_y,Symbol_y,seqname_y,pos_min_y,pos_max_y
0,1,2886,21900206,1.0,A1BG,chr19,58345183.0,58353492.0,2886.0,GRB7,chr17,39737938.0,39747285.0
1,1,80854,21988832,1.0,A1BG,chr19,58345183.0,58353492.0,80854.0,SETD7,chr4,139492974.0,139556219.0
2,1,84236,33961781,1.0,A1BG,chr19,58345183.0,58353492.0,84236.0,RHBDD1,chr2,226800159.0,226999210.0
3,1,148581,33961781,1.0,A1BG,chr19,58345183.0,58353492.0,148581.0,UBE2U,chr1,64203623.0,64267368.0
4,1,5655,33961781,1.0,A1BG,chr19,58345183.0,58353492.0,5655.0,KLK10,chr19,51012739.0,51020097.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
866147,100506658,100506658,10026224,100506658.0,OCLN,chr5,69492547.0,69558104.0,100506658.0,OCLN,chr5,69492547.0,69558104.0
866148,100506742,100506742,"11278723,12097332",100506742.0,CASP12,chr11,104885718.0,104898460.0,100506742.0,CASP12,chr11,104885718.0,104898460.0
866149,100652748,100652748,30033366,100652748.0,TIMM23B,chr10,49942053.0,49974850.0,100652748.0,TIMM23B,chr10,49942053.0,49974850.0
866150,102723631,102723631,32296183,102723631.0,CT45A10,chrX,135881065.0,135893444.0,102723631.0,CT45A10,chrX,135881065.0,135893444.0


In [57]:
import gzip

def leer_secuencias_fasta(ruta_archivo):
    secuencias = {}  # Diccionario para almacenar las secuencias de cada cromosoma
    with gzip.open(ruta_archivo, 'rt') as f:
        nombre_cromosoma = ''
        secuencia_actual = []
        for linea in f:
            if linea.startswith('>'):
                # Procesar la secuencia anterior y almacenarla si hay una secuencia actual
                if nombre_cromosoma and secuencia_actual:
                    secuencias[nombre_cromosoma] = ''.join(secuencia_actual)
                    secuencia_actual = []  # Reiniciar la lista para la próxima secuencia
                nombre_cromosoma = linea.strip()[1:]  # Obtener el nombre del cromosoma
            else:
                # Almacenar la línea de secuencia actual
                secuencia_actual.append(linea.strip())
        
        # Almacenar la última secuencia si existe
        if nombre_cromosoma and secuencia_actual:
            secuencias[nombre_cromosoma] = ''.join(secuencia_actual)
            
    return secuencias

# Lista de nombres de archivos FASTA comprimidos por cromosoma
file_paths = [
    'archivos_fasta_cromosomas/chrM.fa.gz',
    'archivos_fasta_cromosomas/chrX.fa.gz',
    'archivos_fasta_cromosomas/chrY.fa.gz',
    'archivos_fasta_cromosomas/chr22.fa.gz',
    'archivos_fasta_cromosomas/chr21.fa.gz',
    'archivos_fasta_cromosomas/chr20.fa.gz',
    'archivos_fasta_cromosomas/chr19.fa.gz',
    'archivos_fasta_cromosomas/chr18.fa.gz',
    'archivos_fasta_cromosomas/chr17.fa.gz',
    'archivos_fasta_cromosomas/chr16.fa.gz',
    'archivos_fasta_cromosomas/chr15.fa.gz',
    'archivos_fasta_cromosomas/chr14.fa.gz',
    'archivos_fasta_cromosomas/chr13.fa.gz',
    'archivos_fasta_cromosomas/chr12.fa.gz',
    'archivos_fasta_cromosomas/chr11.fa.gz',
    'archivos_fasta_cromosomas/chr10.fa.gz',
    'archivos_fasta_cromosomas/chr9.fa.gz',
    'archivos_fasta_cromosomas/chr8.fa.gz',
    'archivos_fasta_cromosomas/chr7.fa.gz',
    'archivos_fasta_cromosomas/chr6.fa.gz',
    'archivos_fasta_cromosomas/chr5.fa.gz',
    'archivos_fasta_cromosomas/chr4.fa.gz',
    'archivos_fasta_cromosomas/chr3.fa.gz',
    'archivos_fasta_cromosomas/chr2.fa.gz',
    'archivos_fasta_cromosomas/chr1.fa.gz'
]

# Diccionario para almacenar las secuencias de los cromosomas
chromosome_sequences = {}

# Leer las secuencias de cada archivo y almacenarlas en el diccionario
for file_path in file_paths:
    chromosome_name = file_path.split('/')[-1].split('.')[0]
    sequences = leer_secuencias_fasta(file_path)
    chromosome_sequences[chromosome_name] = sequences


In [58]:
def obtener_secuencia(inicio, fin, cromosoma):
    secuencia = chromosome_sequences[cromosoma][cromosoma][inicio-1:fin]
    return secuencia

In [59]:
df_2_inner['pos_min_x'] = df_2_inner['pos_min_x'].astype(int)
df_2_inner['pos_max_x'] = df_2_inner['pos_max_x'].astype(int)
df_2_inner['pos_min_y'] = df_2_inner['pos_min_y'].astype(int)
df_2_inner['pos_max_y'] = df_2_inner['pos_max_y'].astype(int)


C:\Users\mojoy\AppData\Local\Temp\ipykernel_6368\2470863266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_inner['pos_min_x'] = df_2_inner['pos_min_x'].astype(int)
C:\Users\mojoy\AppData\Local\Temp\ipykernel_6368\2470863266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_inner['pos_max_x'] = df_2_inner['pos_max_x'].astype(int)
C:\Users\mojoy\AppData\Local\Temp\ipykernel_6368\2470863266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [69]:
# Crear un diccionario vacío para almacenar los resultados
resultados_diccionario_1 = {}

# Iterar sobre las filas del DataFrame
for index, row in df_2_inner.iterrows():
    # Obtener el valor de la columna 'id'
    id_valor = row['gene_id']
    # Calcular el resultado de la función para la fila actual
    resultado = obtener_secuencia(row['pos_min_x'], row['pos_max_x'], row['seqname_x']).upper()
    # Almacenar el resultado en el diccionario usando el valor de 'id' como clave
    resultados_diccionario_1[id_valor] = resultado



In [70]:
resultados_diccionario_2 = {}

# Iterar sobre las filas del DataFrame
for index, row in df_2_inner.iterrows():
    # Obtener el valor de la columna 'id'
    id_valor = row['interactant_id']
    # Calcular el resultado de la función para la fila actual
    resultado = obtener_secuencia(row['pos_min_y'], row['pos_max_y'], row['seqname_y']).upper()
    # Almacenar el resultado en el diccionario usando el valor de 'id' como clave
    resultados_diccionario_2[id_valor] = resultado


In [71]:
resultados_diccionario_1[1]

'AAACAACCCAATATTTTAAAGTGTGCAAAATATATAAACAGACATTTCATCAAAGATGCTATATAGATGGCAACTAAGCACTTGGGGAAAAAATGCTTAACATCATCAGACATTAAGGAAACGCAAGGAAAACCACTATTAGATATGTCTACATACCTATTAGAATGGCTAAAATAAACTTTTTAAAAAACTAAGCTGGGGCTGGGCGTGGTGGCTCACCCCTGTAATCCCAGCACTTTGGGAGGCTGAGGCGGGTGGATCACTTGAGCTCAGGAGTTGGAGACCAGCCTGGCCACCCAACAAGGTGAAACCCCATCTCTACTAAAAACATAAAAATTAGCTGGGTGTGGTGGTGGGTGCCTGTCAGCCCAGCTACTCAGGAGGCTGAGGCACGAGAATCACTTGAACCCAGGAGGTAGAGGTTGCTGTGAGCCAAGATCACGCCATTGCACTCCAGCCTGGGCAACAAGAGCAAAATTCTGTCTCAAAAACAAATAAACAAGAAAAGGTTTTAAAAATTGATAAACCACTAGCAAGAAAGAGAGAGAGGACACATCAGGCTTGAGAGAAGTGACATCATCATAGACTGTGCAGACATTAAAAGGCATAGGCATATTAGGAACTAGATATTACTAAGGCAGTAAACCAGATCACTAGCAAATTCGTTGAAAAACACAATTCATCAACTATTGGTTGATAATACTGATACGCATCTTCTGTGTGCCAGGGCCTTGAGGCCCTGCTCCAGGCAGAAACTCGGCAGTGGTTGGGAAGGCATCAGATGTCAGATGGCACAAGAGGACTCAGAGCTGAGGGATAATGGGAAAGAACACAGAGGAATCCAGCCATTTCCACAGCGTCCAGCTCTGCTGTGGGAGGCTGGGAACAGCCCAGCACTACCACCCTGGACTGGGAGGACAAGACCACAAAATGCAGCTTCCCTGAACCTCCTCTTGGTGATGGGGTTGATGTGGTCGAGGTAGAGGTGAGTATGTCTGG

In [72]:
resultados_diccionario_2[2886]

'AGTTTCCTTGGGCCTGGAATCTGGACACACAGGGCTCCCCCCCGCCTCTGACTTCTCTGTCCGAAGTCGGGACACCCTCCTACCACCTGTAGAGAAGCGGGAGTGGATCTGAAATAAAATCCAGGAATCTGGGGGTTCCTAGACGGAGCCAGACTTCGGAACGGGTGTCCTGCTACTCCTGCTGGGGCTCCTCCAGGTAAGGGGACCGAGATCGGTCTCAGACGACAGCCTGCCAGGCAACCCCCAGCCGCACCAGGGCCCCAGGCTGTGCCAGTCTCCGGCTGAACTGGCCGGCGGCCCAGGTGGGAGGATTGGAGCCACCTGGCTCTCCCCCTCCCCCGCTCGGTGATGTCAGGCCAGGGGCGGGGGCTGGAGGGGGACAGGTAGTTAAGGGCCTGGCGTCTCCCTCCCTGAAGACGTGGTCCCAGCCGGGTGTCCTGACGCTCGGGGTTCAGGTAAGAACCCAAGCGGGCATTTATCCTTTGGGGGCAGGCACGGCCGGTCTGGAGGCCGGTAGGGAAAGGCTGGGTCTCCTGATGGGTTGCTTGGCCTCCCACTTTTCTCTTTTCTGTCCATTTCCCACCCTGCCCCCTCCTCCCCACCCGCCCTCTTTTCGTCCCCATCCTGGTTTCCAGCGCTCCTTGTTCCCTCCCTGTACACCTCCCTGCAGGCAGCAGCTGAGAATGGGAAGGAAGACCTGATTCTCTCCTCCGACTTGGCTTTCCCGCTGAAACTAGCCCCTCCCCGCACACCCGGGGAGAGCCGCCCTCACTCTCTGGCTCGCCCAGTTTACCCCAGTTTGGGTTTCCCCAAGTCTCTAAGACAGACGTCTCTGCGGGCTGCGGGGAGATGTGGGGAGGGCCCCCTCCACTTTGGAGGGCAGTGAAGGAGAGGGATCCTCTAAATTGTCGAGGCTTCATCTCTCCAGATTGTATGCCCTTCTCAGCAACACCGCCTCCGGCCCTCCGATGGGAAAGTGGAGGCCGGGTGAGACGGGGT